In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

## 3D meshing example

This notebook shows how to

1. Load and visualize a volume
1. Apply image filters to and segment image
1. Generate a 3D surface mesh from the binary image
1. Decimate/simplify a 3D mesh
1. Visualize and save the mesh to gmsh22 format

### Todo:

- Generate 3D volume mesh
- Surface mesh must be 'watertight'
- Constrained Delaunay triangulation to fix surface (do not go outside surface), i.e.
https://notebook.community/daniel-koehn/Theory-of-seismic-waves-II/02_Mesh_generation/4_Tri_mesh_delaunay_yigma_tepe,
https://wias-berlin.de/software/tetgen/1.5/doc/manual/manual002.html#sec7
- For padding, keep tetrahedron if one of the points is inside the boundaries

In [ ]:
from nanomesh.volume import Volume
import pyvista as pv
from skimage import filters
import numpy as np

In [ ]:
vol = Volume.load('sample_data.npy')

vol.image = vol.image[50:100, 50:100, 50:100]

vol_gauss = vol.apply(filters.gaussian, sigma=5)
thresh = filters.threshold_li(vol_gauss.image)
seg_vol = vol_gauss.apply(np.digitize, bins=[thresh])

seg_vol.show_slice()

### Generate 3d tetragonal mesh

In [ ]:
%%time

from nanomesh.mesh3d import generate_3d_mesh

mesh = generate_3d_mesh(
    seg_vol.image,
    pad_width=20,
    point_density=1/500,
    step_size=1,
    res_kmeans=1.0,
    n_faces=5000,
)

pv.plot_itk(mesh)

In [ ]:
from nanomesh.mesh3d import show_submesh

show_submesh(mesh, index=100)